In [1]:
from numba import jit, cuda
import numpy as np
from timeit import default_timer as timer 

In [2]:
n = 100000000
a = np.ones(n, dtype = np.float64)
a.nbytes/1e6

800.0

In [3]:
# normal function to run on cpu
def func(x):
    a=x.copy()
    for i in range(a.shape[0]):
        a[i]+= 1
    return a

# function optimized to run on gpu 
@jit(target_backend='cuda')
def func_numba_cuda(x):
    a=x.copy()
    for i in range(a.shape[0]):
        a[i]+= 1
    return a

@jit 
def func_numba(x):
    a=x.copy()
    for i in range(a.shape[0]):
        a[i]+= 1
    return a

@jit 
def func_numba_nonvecable(x):
    a=x.copy()
    asum=0.0
    for i in range(a.shape[0]):
        a[i]+= 1
        asum += a[i]
    return asum

def func_nonvecable(x):
    a=x.copy()
    asum=0.0
    for i in range(a.shape[0]):
        a[i]+= 1
        asum += a[i]
    return asum

In [4]:
start = timer()
b=func(a)
print("numpy:                 ", timer()-start, " result= ",b.sum()) 
start = timer()
b=func(a)
print("numpy:                 ", timer()-start, " result= ",b.sum()) 

start = timer()
b=func_nonvecable(a)
print("numpy nonvecable:       ", timer()-start, " result= ",b)
start = timer()
b=func_nonvecable(a)
print("numpy nonvecable:       ", timer()-start, " result= ",b)

start = timer()
b=func_numba_nonvecable(a)
print("numba jit nonvecable:   ", timer()-start, " result= ",b)
start = timer()
b=func_numba_nonvecable(a)
print("numba jit nonvecable:   ", timer()-start, " result= ",b)

start = timer()
b=func_numba(a)
print("numba jit:              ", timer()-start, " result= ",b.sum())
start = timer()
b=func_numba(a)
print("numba jit:              ", timer()-start, " result= ",b.sum())
start = timer()
b=func_numba(a)
print("numba jit:              ", timer()-start, " result= ",b.sum())
   
start = timer()
b=func_numba_cuda(a)
print("numba jit backend cuda: ", timer()-start, " result= ",b.sum())
start = timer()
b=func_numba_cuda(a)
print("numba jit backend cuda: ", timer()-start, " result= ",b.sum())

#None of the above seem to utilize GPU as evidenced by nvidia-smi
#The jit ones need a warmup (perhaps to compile)
#numpy:                  21.35097421798855  result=  200000000.0
#numpy:                  20.661394510883838  result=  200000000.0
#numpy nonvecable:        35.62613185006194  result=  200000000.0
#numpy nonvecable:        35.486699688015506  result=  200000000.0
#numba jit nonvecable:    0.6851420938037336  result=  200000000.0
#numba jit nonvecable:    0.21572539093904197  result=  200000000.0
#numba jit:               0.2543515469878912  result=  200000000.0
#numba jit:               0.16620380710810423  result=  200000000.0
#numba jit:               0.16279309103265405  result=  200000000.0
#numba jit backend cuda:  0.25583608797751367  result=  200000000.0
#numba jit backend cuda:  0.17059857910498977  result=  200000000.0

numpy:                  21.7702771241311  result=  200000000.0
numpy:                  21.51454497780651  result=  200000000.0
numpy nonvecable:        35.72810862399638  result=  200000000.0
numpy nonvecable:        34.885415617143735  result=  200000000.0
numba jit nonvecable:    0.6842681178823113  result=  200000000.0
numba jit nonvecable:    0.21199987595900893  result=  200000000.0
numba jit:               0.2455444810912013  result=  200000000.0
numba jit:               0.15868577198125422  result=  200000000.0
numba jit:               0.16060847020708025  result=  200000000.0
numba jit backend cuda:  0.24505614885129035  result=  200000000.0
numba jit backend cuda:  0.1603886450175196  result=  200000000.0


In [5]:
import dace
dace_func = dace.program(auto_optimize=True)(func)

start = timer()
b=dace_func(a)
print("dace:               ", timer()-start, " result= ",b.sum())
start = timer()
b=dace_func(a)
print("dace:               ", timer()-start, " result= ",b.sum())
start = timer()
b=dace_func(a)
print("dace:               ", timer()-start, " result= ",b.sum())
#dace:                1.3049751319922507  result=  200000000.0
#dace:                0.22182884812355042  result=  200000000.0
#dace:                0.21486610011197627  result=  200000000.0


dace:                1.4020556551404297  result=  200000000.0
dace:                0.22581780282780528  result=  200000000.0
dace:                0.23822007817216218  result=  200000000.0


In [6]:
#This seems to utilize GPU,
# the python process appears in nvidiai-smi
# the memory usage on device 0 increases by 800MiB everytime the cell runs!
#Before run | N/A   31C    P0              28W / 250W |    125MiB / 32768MiB |      0%      Default |
#After  run | N/A   31C    P0              38W / 250W |   1201MiB / 32768MiB |      0%      Default |
# How to clean it up?
#
#Also, it needs warmup runs

@cuda.jit
def func_numba_cuda_jit(a):
    pos = cuda.grid(1)
    if pos < a.size: # Check array boundaries
        a[pos]+= 1
    #return a

d_a = cuda.to_device(a) # upload a to the GPU
threadsperblock = 256
blockspergrid = (d_a.size + (threadsperblock - 1)) // threadsperblock

start = timer()
func_numba_cuda_jit[blockspergrid, threadsperblock](d_a)
print("numba cuda jit:     ", timer()-start, " result= ")
start = timer()
func_numba_cuda_jit[blockspergrid, threadsperblock](d_a)
print("numba cuda jit:     ", timer()-start, " result= ")
start = timer()
func_numba_cuda_jit[blockspergrid, threadsperblock](d_a)
print("numba cuda jit:     ", timer()-start, " result= ")
start = timer()
func_numba_cuda_jit[blockspergrid, threadsperblock](d_a)
print("numba cuda jit:     ", timer()-start, " result= ")

#numba cuda jit:      0.2099861961323768  result= 
#numba cuda jit:      0.00035113305784761906  result= 
#numba cuda jit:      9.70128457993269e-05  result= 
#numba cuda jit:      9.366683661937714e-05  result= 
#numba cuda jit:      0.2675072008278221  result= 
#numba cuda jit:      0.000666347099468112  result= 
#numba cuda jit:      0.0001106590498238802  result= 
#numba cuda jit:      9.953812696039677e-05  result= 

numba cuda jit:      0.33763771899975836  result= 
numba cuda jit:      0.000697998097166419  result= 
numba cuda jit:      0.00012038717977702618  result= 
numba cuda jit:      0.00010664109140634537  result= 


In [7]:
from numba import jit
import numpy as np

def go_slow(a):
    for i in range(a.shape[0]):
        a[i] += 1
    return a

@jit
def go_fast(a):
    for i in range(a.shape[0]):
        a[i] += 1
    return a

In [8]:
from timeit import timeit

# jit warmup
go_fast(a)
go_slow(a)

ts = timeit(lambda: go_slow(a), number=2)
tf = timeit(lambda: go_fast(a), number=2)
print(f'slow: {ts*1000:.2f}ms fast: {tf*1000:.2f}ms speedup: {ts/tf:.1f}x')

slow: 41971.45ms fast: 87.89ms speedup: 477.6x


In [9]:
import numpy as np
from numba import cuda

@cuda.jit
def increment_by_one(a):
    pos = cuda.grid(1)
    if pos < a.size:  # Check array boundaries
        a[pos] += 1

d_a = cuda.to_device(a) # upload a to the GPU
threadsperblock = 256
blockspergrid = (d_a.size + (threadsperblock - 1)) // threadsperblock
increment_by_one[blockspergrid, threadsperblock](d_a)

In [10]:
from timeit import timeit

# warmup
go_fast(a)
go_slow(a)
increment_by_one[blockspergrid, threadsperblock](d_a)

tf = timeit(lambda: go_fast(a), number=2)
ts = timeit(lambda: go_slow(a), number=2)
tg = timeit(lambda: increment_by_one[blockspergrid, threadsperblock](d_a),
    number=2)

print(f'slow: {ts*1000:.2f}ms fast: {tf*1000:.2f}ms speedup: {ts/tf:.1f}x')
print(f'fast: {tf*1000:.2f}ms cuda: {tg*1000:.2f}ms speedup: {tf/tg:.1f}x')
print(f'slow: {ts*1000:.2f}ms cuda: {tg*1000:.2f}ms speedup: {ts/tg:.1f}x')
#slow: 42823.86ms fast: 86.61ms speedup: 494.5x
#fast: 86.61ms cuda: 0.78ms speedup: 110.5x
#slow: 42823.86ms cuda: 0.78ms speedup: 54619.5x

slow: 42651.39ms fast: 87.54ms speedup: 487.2x
fast: 87.54ms cuda: 0.86ms speedup: 102.3x
slow: 42651.39ms cuda: 0.86ms speedup: 49854.6x


In [11]:
start = cuda.event()
end = cuda.event()
start.record()
increment_by_one[blockspergrid, threadsperblock](d_a)
end.record()
end.synchronize()
elapsed = start.elapsed_time(end)
print(f'Event timing: {elapsed:.3f}ms')


Event timing: 2.098ms


In [12]:
def inc(blockspergrid, threadsperblock, a):
    increment_by_one[blockspergrid, threadsperblock](a)
    cuda.synchronize()

tg = timeit(lambda: inc(blockspergrid, threadsperblock, d_a), number=2)
print(f'fast: {tf*1000:.2f}ms cuda: {tg*1000:.2f}ms speedup: {tf/tg:.1f}x')
#fast: 86.61ms cuda: 4.12ms speedup: 21.0x

fast: 87.54ms cuda: 4.22ms speedup: 20.8x
